### 代码逻辑总结

1. 导入必要的库和设置超参数
2. 加载FashionMNIST数据集并创建数据加载器
3. 探索数据形状并确定输入输出维度
4. 定义一个简单的两层神经网络模型
5. 定义训练函数，包含前向传播、损失计算、反向传播和性能监控
6. 实例化模型并开始训练

### **1. 导入库（关键库说明）**

```python
import torch  # 核心张量计算库
import torch.nn as nn  # 神经网络层和损失函数
import torch.optim as optim  # 优化算法（SGD/Adam）
from torch.nn import functional as F  # 函数式操作（如relu, softmax）
from torch.utils.data import DataLoader, TensorDataset  # 数据批量加载
import torchvision  # 计算机视觉工具
import torchvision.transforms as transforms  # 图像预处理
```

---

### **2. 超参数设置**

```python
lr = 0.15      # 学习率（控制参数更新步长）
gamma = 0.8    # SGD动量参数（加速收敛）
epochs = 3     # 训练轮数
bs = 128       # 批大小（影响内存和训练稳定性）
```

---

### **3. 数据加载与预处理**

```python
# 加载FashionMNIST数据集
mnist = torchvision.datasets.FashionMNIST(
    root='D:\数据集',  # 存储路径
    train=True,        # 训练集
    download=True,     # 自动下载
    transform=transforms.ToTensor()  # 转为张量并归一化到[0,1]
)

# 创建DataLoader
batchdata = DataLoader(mnist, batch_size=bs, shuffle=True)  # 打乱数据

# 检查数据形状
for x, y in batchdata:
    print(x.shape)  # torch.Size([128, 1, 28, 28]) (batch,通道,高,宽)
    print(y.shape)  # torch.Size([128]) (标签)
    break

# 计算输入/输出维度
input_ = mnist.data[0].numel()  # 28*28=784 (展平后的特征数)
output_ = len(mnist.targets.unique())  # 10 (类别数)
```

---

### **4. 神经网络定义**

```python
class Model(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features, 128, bias=False)  # 输入层→隐藏层
        self.output = nn.Linear(128, out_features, bias=False)   # 隐藏层→输出层
    
    def forward(self, x):
        x = x.view(-1, 28*28)          # 展平图像 [batch, 784]
        sigma1 = torch.relu(self.linear1(x))  # ReLU激活
        sigma2 = F.log_softmax(self.output(sigma1), dim=1)  # 输出概率对数
        return sigma2
```

**关键点**：

- `view(-1, 28*28)`：-1自动计算batch大小
- `log_softmax` + `NLLLoss` = 交叉熵损失的高效实现

---

### **5. 训练函数（核心逻辑）**

```python
def fit(net, batchdata, lr=0.01, epochs=5, gamma=0):
    criterion = nn.NLLLoss()  # 负对数似然损失
    opt = optim.SGD(net.parameters(), lr=lr, momentum=gamma)  # 带动量的SGD
    
    samples = correct = 0
    for epoch in range(epochs):
        for batch_idx, (x, y) in enumerate(batchdata):
            y = y.view(-1)  # 确保标签形状为[batch]
            
            # 前向传播
            sigma = net(x)
            loss = criterion(sigma, y)
            
            # 反向传播
            opt.zero_grad()  # 清零梯度（防止累积）
            loss.backward()  # 自动微分
            opt.step()       # 参数更新
            
            # 统计准确率
            yhat = torch.max(sigma, 1)[1]  # 预测类别
            correct += (yhat == y).sum().item()
            samples += x.size(0)
            
            # 打印进度
            if (batch_idx+1) % 125 == 0:
                print(f"Epoch {epoch+1}: [{samples}/{epochs*len(batchdata.dataset)}] "
                      f"Loss: {loss.item():.4f}, Acc: {100*correct/samples:.2f}%")
```

---

### **6. 模型训练**

```python
torch.manual_seed(420)  # 固定随机种子（保证可重复性）
net = Model(input_, output_)  # 初始化模型
fit(net, batchdata, lr=lr, epochs=epochs, gamma=gamma)  # 开始训练
```

---

### **训练方法系统总结**

| **步骤**        | **关键操作**                                                 | **作用**                             |
| --------------- | ------------------------------------------------------------ | ------------------------------------ |
| **1. 数据准备** | `DataLoader` + `transforms`                                  | 实现批量加载、数据打乱和预处理       |
| **2. 模型定义** | 继承`nn.Module`，定义`__init__`和`forward`                   | 构建网络架构（如全连接层、激活函数） |
| **3. 训练配置** | 选择损失函数（如`NLLLoss`）、优化器（如`SGD`）、设置超参数（lr/epochs等） | 确定训练策略和收敛目标               |
| **4. 训练循环** | 交替执行：前向传播 → 计算损失 → 反向传播 → 参数更新          | 通过梯度下降优化模型参数             |
| **5. 监控指标** | 计算并打印损失和准确率                                       | 评估模型性能，调试训练过程           |
| **6. 优化技巧** | 使用动量（momentum）、学习率调整、随机种子固定               | 加速收敛，提高结果可复现性           |

---

### 

# 涉及的知识点

1. PyTorch基础



   - 张量操作（`torch.Tensor`）
   - 自动微分（`autograd`）
   - GPU加速（虽然代码中没有显式使用）

2. 神经网络构建

   

   - `nn.Module`基类
   - 线性层（`nn.Linear`）
   - 激活函数（`ReLU`）
   - 损失函数（`NLLLoss`）

3. 优化算法

   

   - 随机梯度下降（`SGD`）
   - 动量（`momentum`）

4. 数据处理

   

   - 数据集加载（`torchvision.datasets`）
   - 数据预处理（`transforms`）
   - 批量加载（`DataLoader`）

5. 训练流程

   

   - 前向传播
   - 损失计算
   - 反向传播
   - 参数更新
   - 准确率计算

6. 模型评估

   

   - 训练过程中的损失监控
   - 准确率计算

7. 可复现性

   

   - 随机种子设置（`torch.manual_seed`）